Importing the Dependencies

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data Collection and Processing

In [25]:
heart_data = pd.read_csv('heart_disease_data.csv')

In [26]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Bu veri, kalp hastalığı tahmini yapmak için kullanılan bir veri kümesinden alınmış. Aşağıdaki değişkenler (özellikler) ve hedef değişkeni (target) hakkında genel bir değerlendirme yapılabilir:

### Özelliklerin Açıklaması:
1. **age**: Yaş (ör. 63, 37, 41, vb.).
2. **sex**: Cinsiyet (1: Erkek, 0: Kadın).
3. **cp**: Göğüs ağrısı tipi (0-3 arasında kategorik değerler; 3 en şiddetli).
4. **trestbps**: İstirahat halindeki kan basıncı (mmHg cinsinden).
5. **chol**: Serum kolesterol (mg/dL).
6. **fbs**: Açlık kan şekeri (1: >120 mg/dL, 0: <=120 mg/dL).
7. **restecg**: Dinlenme elektrokardiyografik sonuçları (0-2 arasında kategorik değerler).
8. **thalach**: Maksimum kalp atış hızı.
9. **exang**: Egzersiz kaynaklı anjina (1: Evet, 0: Hayır).
10. **oldpeak**: Egzersizden sonra ST depresyonu (ST segmentindeki değişim).
11. **slope**: ST segmentinin eğimi (0-2 arasında kategorik değerler).
12. **ca**: Floreskopi ile görülen majör damar sayısı (0-3 arasında).
13. **thal**: Talasemi durumu (1-3 arasında kategorik değerler).
14. **target**: Hedef değişken (1: Kalp hastalığı var, 0: Kalp hastalığı yok).

### Gözlem:
- **Yaş ve Cinsiyet**: Veri, hem kadın hem erkek bireylerden oluşuyor. İlk örnekte yaşlar 37-63 arasında değişiyor.
- **Göğüs Ağrısı Türü (cp)**: Şiddetli göğüs ağrısı (3) olan bireyler daha fazla olabilir, bu da kalp hastalığı riskinin yüksek olduğunu gösterebilir.
- **Kan Basıncı ve Kolesterol**: Trestbps ve chol değerleri, kişinin kardiyovasküler sağlığı hakkında önemli ipuçları verir.
- **Egzersiz Anjinası (exang)**: İlk 5 bireyin çoğunda egzersiz sırasında anjina bulunmuyor (0).
- **Hedef Değişken**: İlk 5 bireyin hepsinde hedef değeri 1, yani kalp hastalığı mevcut. Bu, kalp hastalığı olan bireylerin özelliklerini analiz etmek için bir alt küme olabilir.

In [27]:
heart_data.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [28]:
# number of rows and columns in the dataset
heart_data.shape

(303, 14)

In [29]:
# getting some info about the data
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [30]:
# checking for missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [31]:
# statistical measures about the data
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [32]:
# checking the distribution of Target Variable
heart_data['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

1 --> Defective Heart

0 --> Healthy Heart

Splitting the Features and Target

In [33]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [34]:
print(X)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [35]:
print(Y)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64


Splitting the Data into Training data & Test Data

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [37]:
print(X.shape, X_train.shape, X_test.shape)

(303, 13) (242, 13) (61, 13)


Model Training

Logistic Regression

In [38]:
model = LogisticRegression()

In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)  # Increase from the default 100
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [40]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [41]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.8553719008264463


In [42]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [43]:
print('Accuracy on Test data : ', test_data_accuracy)

Accuracy on Test data :  0.8032786885245902


Building a Predictive System

#### alttaki kod icin aciklama
Bu satırın amacı, `input_data` adlı NumPy dizisini, modelinizin eğitildiği `X` DataFrame ile aynı sütun adlarına sahip bir **pandas DataFrame** haline dönüştürmektir. Bu işlem, modelin sütun adlarını beklediği durumlarda uyumluluk sağlamak için yapılır.

### **Detaylı Açıklama:**

#### **1. `input_data`**
`input_data` bir NumPy dizisidir. Örneğin:
```python
input_data = (62, 0, 0, 140, 268, 0, 0, 160, 0, 3.6, 0, 2, 2)
```
Bu, modelin eğitildiği özellikleri içerir ama bu verilere sütun adları eklenmemiştir.

---

#### **2. `pd.DataFrame([input_data], columns=X.columns)`**
Bu satır şunları yapar:
- **`pd.DataFrame()`**: Yeni bir DataFrame oluşturur.
- **`[input_data]`**: `input_data`'yı bir listeye alarak DataFrame'e uygun bir şekle getirir. `[input_data]` bir 2D yapı sağlar çünkü pandas DataFrame, iki boyutlu verilerle çalışır.
- **`columns=X.columns`**: `X` DataFrame'inin sütun adlarını kullanır. `X` modelin eğitildiği özellikleri içeriyordu ve bu özelliklerin adları `X.columns` ile alınır.

Bu işlem sonucunda, `input_data` bir pandas DataFrame'e dönüşür ve aynı zamanda `X` ile uyumlu sütun adlarına sahip olur.

---

#### **Örnek:**

Varsayalım, `X.columns` şunları içeriyor:
```python
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
```

Ve `input_data` şu:
```python
(62, 0, 0, 140, 268, 0, 0, 160, 0, 3.6, 0, 2, 2)
```

Bu kodu çalıştırdığınızda, aşağıdaki gibi bir DataFrame oluşur:
```python
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  ca  thal
0   62    0   0       140   268    0        0      160      0      3.6      0   2     2
```

Artık bu DataFrame, sütun adlarıyla birlikte tahmin için kullanılabilir.

---

#### **Neden Gerekli?**
- Modelinizi eğitirken pandas DataFrame kullandığınız için model sütun adlarına göre çalışır.
- Tahmin yaparken de sütun adları bekler, aksi takdirde uyarı verir veya hata oluşabilir. Bu satır, modelin beklediği şekilde giriş verilerini düzenler.


In [47]:
# Tahmin verisini DataFrame'e dönüştürün
input_data_df = pd.DataFrame([input_data], columns=X.columns)

# Tahmin yapın
prediction = model.predict(input_data_df)

# Tahmin sonucunu yazdırın
if prediction[0] == 0:
    print('The Person does not have a Heart Disease')
else:
    print('The Person has Heart Disease')

The Person has Heart Disease


## Bu kod, bir makine öğrenmesi modelini kullanarak bir bireyin kalp hastalığı olup olmadığını tahmin etmek için tasarlanmıştır. Örnek bir girdi verisi (yaş, cinsiyet, kan basıncı, kolesterol vb.) NumPy dizisine dönüştürülüp modele uygun şekilde yeniden şekillendirilir. Eğitimli model, bu girdiyle bir tahmin yapar ve sonuç, bireyin kalp hastalığı taşıyıp taşımadığını belirten bir mesaj olarak yazdırılır (0: Kalp hastalığı yok, 1: Kalp hastalığı var).